In [1]:
from generate_3_sat_constraints import solve_with_solver, generate_3_sat_constraints
from real_constraint_utils import solve_smt_formula, cnf_to_smt_over_reals
from pysat.formula import CNF
import pandas as pd
from tqdm import tqdm
from pysat.solvers import Minisat22, Glucose3, Lingeling

# Generate CNF

In [2]:
generate_3_sat_constraints(10, formula_dir="/Users/mihajlobulesnij/Documents/system/RAI/project/DRL_DGM/constraints_generated/sat_constraints")

Generating formulas and benchmarking...


Variables: 100%|██████████| 1/1 [00:12<00:00, 12.97s/it]


✅ Benchmark complete. Results saved to '/Users/mihajlobulesnij/Documents/system/RAI/project/DRL_DGM/constraints_generated/sat_constraints/sat_formulas.csv', formulas saved in '/Users/mihajlobulesnij/Documents/system/RAI/project/DRL_DGM/constraints_generated/sat_constraints/'


# Convert to constraints over Reals

In [6]:
dataset = pd.read_csv("constraints_generated/sat_constraints/sat_formulas.csv")
for filename in tqdm(dataset["filename"]):
    time_mesured_buf = []
    cnf = CNF(from_file=f"constraints_generated/sat_constraints/{filename}")
    cnf_to_smt_over_reals(cnf, filename = "constraints_generated/smt_constraints_random/"+filename[:-4], random_v = True)

100%|██████████| 130/130 [00:00<00:00, 222.16it/s]


Generate data that satisfy constraints with DRL

In [7]:
dataset = pd.read_csv("constraints_generated/sat_constraints/sat_formulas.csv")
for filename in tqdm(dataset["filename"]):
    time_mesured_buf = []
    cnf = CNF(from_file=f"constraints_generated/sat_constraints/{filename}")
    cnf_to_smt_over_reals(cnf, filename = "constraints_generated/smt_constraints/"+filename[:-4], random_v = False)

100%|██████████| 130/130 [00:00<00:00, 228.79it/s]


# Generating Dataset that satisfy constraints

In [3]:
import os
import subprocess
import pandas as pd
from tqdm import tqdm
base_path = "/Users/mihajlobulesnij/Documents/system/RAI/Constraints_complexity/smt_formulas_random"
seeds = [1, 2, 3]
python_script = "sample_random_data.py"

info_cnf = pd.read_csv("/Users/mihajlobulesnij/Documents/system/RAI/Constraints_complexity/sat_formulas.csv")
sats = info_cnf.Sat
filenames = info_cnf.filename
commands = []

for file_name, sat in zip(filenames, sats):
    if sat:
        full_constraint_path = os.path.join(base_path, file_name[:-4]+".txt")
        
        for seed in seeds:
            cmd = f'/opt/homebrew/Caskroom/miniconda/base/envs/drl_env/bin/python {python_script} --seed {seed} --constraint_path "{full_constraint_path}" --save_path data_generated/data_smt_random'
            commands.append(cmd)

In [4]:
for i, cmd in tqdm(enumerate(commands, 1)):
    print(f"\n▶️ Running command {i}/{len(commands)}:\n{cmd}\n")
    
    result = subprocess.run(cmd, shell=True)

    if result.returncode != 0:
        print(f"❌ Command failed with exit code {result.returncode}:")
        print(cmd)
        break
    else:
        print(f"✅ Completed {i}/{len(commands)}\n")

1635


100%|███████████████████████████████████████| 357/357 [00:01<00:00, 311.98it/s]


561


100%|██████████████████████████████████████| 200/200 [00:00<00:00, 2052.31it/s]


133
27
5
0
--------------------------------------------------------------------------------


...

173
42
10
2
--------------------------------------------------------------------------------


100%|█████████████████████████████████████████| 2/2 [00:00<00:00, 83055.52it/s]


tensor([[-0.1710, -0.4392, -0.4078,  0.2520,  0.2060,  0.3430, -0.4502, -0.1340,
         -0.2080,  0.3340],
        [-0.1710, -0.4711, -0.4250,  0.2520,  0.2060,  0.3430, -0.4475, -0.1340,
         -0.2080,  0.3340],
        [-0.1710, -0.4688, -0.4083,  0.2520,  0.2060,  0.3430, -0.4564, -0.1340,
         -0.2080,  0.3340]])


276it [54:42, 11.89s/it]

✅ Completed 276/276



### Training

In [5]:
import os
import subprocess
import pandas as pd
from tqdm import tqdm
base_path = "/Users/mihajlobulesnij/Documents/system/RAI/Constraints_complexity/smt_formulas"
seeds = [1, 2, 3]
python_script = "sample_random_data.py"

info_cnf = pd.read_csv("/Users/mihajlobulesnij/Documents/system/RAI/Constraints_complexity/sat_formulas.csv")
sats = info_cnf.Sat
filenames = info_cnf.filename
commands = []

for file_name, sat in zip(filenames, sats):
    if sat:
        full_constraint_path = os.path.join(base_path, file_name[:-4]+".txt")
        for seed in seeds:
            cmd = f'/opt/homebrew/Caskroom/miniconda/base/envs/drl_env/bin/python {python_script} --seed {seed} --constraint_path "{full_constraint_path}" --save_path data_generated/data_smt'
            commands.append(cmd)

In [6]:
for i, cmd in tqdm(enumerate(commands[143:], 1)):
    print(f"\n▶️ Running command {i}/{len(commands)}:\n{cmd}\n")
    
    result = subprocess.run(cmd, shell=True)

    if result.returncode != 0:
        print(f"❌ Command failed with exit code {result.returncode}:")
        print(cmd)
        break
    else:
        print(f"✅ Completed {i}/{len(commands)}\n")


0it [00:00, ?it/s]


▶️ Running command 1/276:
/opt/homebrew/Caskroom/miniconda/base/envs/drl_env/bin/python sample_random_data.py --seed 3 --constraint_path "/Users/mihajlobulesnij/Documents/system/RAI/Constraints_complexity/smt_formulas/n10_r2.8_i2.txt" --save_path data_generated/data_smt

All random seeds set to 3
 *** ALL CONSTRAINTS ***
32
75
425


100%|██████████████████████████████████████| 453/453 [00:00<00:00, 1117.84it/s]


727


100%|███████████████████████████████████████| 225/225 [00:00<00:00, 641.77it/s]


...

173
42
10
2
--------------------------------------------------------------------------------


100%|█████████████████████████████████████████| 2/2 [00:00<00:00, 64527.75it/s]


tensor([[ 0.0500, -0.4392, -0.4078,  0.0500,  0.0500,  0.0500, -0.4502,  0.0500,
          0.0500,  0.0500],
        [ 0.0500, -0.4711, -0.4250,  0.0500,  0.0500,  0.0500, -0.4475,  0.0500,
          0.0500,  0.0500],
        [ 0.0500, -0.4688, -0.4083,  0.0500,  0.0500,  0.0500, -0.4564,  0.0500,
          0.0500,  0.0500]])


133it [33:45, 15.23s/it]

✅ Completed 133/276

